In [1]:
import os
import json
import pandas
import numpy as np
import pickle
from sklearn import preprocessing
import tensorflow as tf
import matplotlib.pyplot as plt

% matplotlib inline

In [2]:
Ned = []
Med = []
NRd = []
MRd = []
fck = []
As = []
bh = []
a = []
xu = []
l0 = []
listdir = os.listdir("../../data_set/buckling")


for file in listdir:
    f = open("../../data_set/buckling/" + file)
    d = json.loads(f.read())
    f.close()

    for o in d:
        Ned.append(o["Ned"])
        Med.append(o["Med"])
        NRd.append(o["NRd"])
        MRd.append(o["MRd"])
        fck.append(o["fck"])
        As.append(o["As"])
        bh.append(o["bh"])
        a.append(o["a"])
        xu.append(o["xu"])
        l0.append(o["l0"])

rho = []
for i in range(len(As)):
    rho.append(As[i] / bh[i]**2)

In [3]:
df = pandas.DataFrame({
        "l0": l0,
        "Ned": Ned,
        "MRd": MRd,
        "fck": fck,
        "bh": bh,
        "rho": rho,
    }
    )
print(df.shape)
df = df[df["fck"] == 20]
print(df.shape)
df.head(5)

(9766650, 6)
(3158400, 6)


,MRd,Ned,bh,fck,l0,rho
0,37582574,100000,250,20,5400,0.004800
3,41767891,100000,275,20,5400,0.003967
6,46007825,100000,300,20,5400,0.003333
9,50355745,100000,325,20,5400,0.002840
12,54748450,100000,350,20,5400,0.002449


In [4]:
# Scale all features by their mean and std-dev to value 0 - 1. This is the correct one.

feat_df = df[["MRd", "rho", "Ned", "l0"]]
res_df = df[["bh"]]

scale_feat = preprocessing.MinMaxScaler()
scale_feat.fit(feat_df)
print(scale_feat.transform(feat_df))

scale_res = preprocessing.MinMaxScaler()
scale_res.fit(res_df)

try:
    with open("rescaled_values_fck20.pkl", "rb") as f:
        df = pickle.load(f)
        
except FileNotFoundError:
    df = df.sample(frac=1)
    with open("rescaled_values_fck20.pkl", "wb") as f:
        pickle.dump(df, f)

[[ 0.          0.1088725   0.          0.34693878]
 [ 0.00463363  0.0877827   0.          0.34693878]
 [ 0.00932774  0.07174218  0.          0.34693878]
 ..., 
 [ 0.49945671  0.01866161  0.31010453  0.79591837]
 [ 0.49289762  0.01210865  0.31010453  0.79591837]
 [ 0.49883828  0.00842993  0.31010453  0.79591837]]


In [5]:
split = int(1e6)
df_train = df[:split]
df_test = df[split:]
df_train.head(5)

print(df_train.shape, df_test.shape)

(1000000, 6) (2158400, 6)


In [6]:
train_feat = scale_feat.transform(df_train[["MRd", "rho", "Ned", "l0"]])
train_label = df_train["bh"].as_matrix()
test_feat = scale_feat.transform(df_test[["MRd", "rho", "Ned", "l0"]])
test_label = df_test["bh"].as_matrix()

print("Training features:\n MRd; rho, Ned, l0\n")
print(train_feat[:5])

print("\nTraining labels:\n bh\n")
print(train_label[:5])


print("\nTraining data:", train_feat.shape, train_label.shape)
print("\nTesting data:", test_feat.shape, test_label.shape)



Training features:
 MRd; rho, Ned, l0

[[ 0.35028971  0.03458409  0.12543554  0.        ]
 [ 0.29641559  0.2642502   0.09059233  0.46938776]
 [ 0.58011533  0.00402416  0.60278746  0.28571429]
 [ 0.44147613  0.19988552  0.74912892  0.93877551]
 [ 0.06810538  0.05925889  0.43902439  0.42857143]]

Training labels:
 bh

[675 400 675 450 325]

Training data: (1000000, 4) (1000000,)

Testing data: (2158400, 4) (2158400,)


In [8]:
X = train_feat
y = train_label



In [34]:
graph = tf.Graph()
subset = int(10e3)
n_input = 4  # n variables
n_hidden_1 = 10
n_hidden_2 = n_hidden_1
n_output = 1

with graph.as_default():
    tf_train_feat = tf.placeholder(tf.float32, shape=(subset, n_input))
    tf_train_label = tf.placeholder(tf.float32, shape=(subset))
    
    """
    Weights and biases. The weights matix' columns will be the output vector.
    
    * ndarray([rows, columns])
    * ndarray([in, out])
    
    tf.placeholder(None) and tf.placeholder([None, 3]) means that the row's size is not set. In the second 
    placeholder the columns are prefixed at 3.
    """
    W = {
        "layer_1": tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
        "layer_2": tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
        "layer_3": tf.Variable(tf.truncated_normal([n_hidden_2, n_output]))
    }
    
            
    b = {
        "layer_1": tf.Variable(tf.zeros([n_hidden_1])),
        "layer_2": tf.Variable(tf.zeros([n_hidden_2])),
        "layer_3": tf.Variable(tf.zeros([n_output]))
    }
    
    def computation(X):
        layer_1 = tf.nn.relu(tf.matmul(X, W["layer_1"]) + b["layer_1"])
        layer_2 = tf.nn.relu(tf.matmul(layer_1, W["layer_2"]) + b["layer_2"])
        layer_3 = tf.nn.relu(tf.matmul(layer_2, W["layer_3"]) + b["layer_3"])
        return layer_3
    
    prediction = computation(tf_train_feat)
    
    loss = tf.reduce_mean(tf.pow(tf_train_label - prediction, 2))
    